In [18]:
from music21 import *
import glob

In [49]:
n = note.Note("D#3")
n.duration.type = 'half'
n.pitch.midi

51

In [11]:
a = environment.Environment()

In [25]:
configure.run()


______________________________________________________________________________ 
  
Welcome the music21 Configuration Assistant. You will be guided through a 
number of questions to install and setup music21. Simply pressing return at a 
prompt will select a default, if available. 

You may run this configuration again at a later time by running 
music21/configure.py. 
  
______________________________________________________________________________ 
  
The BSD/LGPL licensed music21 software is distributed with a corpus of encoded 
compositions which are distributed with the permission of the encoders (and, 
where needed, the composers or arrangers) and where permitted under United 
States copyright law. Some encodings included in the corpus may not be used 
for commercial uses or have other restrictions: please see the licenses 
embedded in individual compositions or directories for more details. 
  
In addition to the corpus distributed with music21, other pieces are not 
included in

In [24]:
b = corpus.parse('bach/bwv66.6')

In [27]:
PATH = "/home/krzysztof/ml/deepbach/bach"

files = glob.glob(PATH + '/**/*.mid', recursive=True)

for file in files:
    mid = midi.MidiFile(file)
    find_base(mid)
    for i, track in enumerate(mid.tracks):
        print('{} : Track {}: {}'.format(file, i, track.name))
        _, global_min_note, global_max_note = track_to_list(track, base, global_min_note, global_max_note)

TypeError: __init__() takes 1 positional argument but 2 were given

In [45]:
s = converter.parse('/home/krzysztof/ml/deepbach/bach/aof/can1.mid')

In [46]:
k = s.analyze('key')
        #if "major" in k: print("MAJOR")
        #i = interval.Interval(k.tonic, pitch.Pitch('C'))
        #print(k, i)
        #sNew = s.transpose(i)
        #kNew = sNew.analyze("key")
        #print("New key: ", kNew)

In [47]:
if "major" in k.name: print("minor")

In [48]:
l, global_min_note, global_max_note, tracks, maxk = track_to_list(s, global_min_note, global_max_note)

Voice 0


AttributeError: 'Note' object has no attribute 'midi'

In [ ]:
l

In [51]:
from music21 import *
import glob
import numpy as np
import pandas as pd
import os
from os.path import basename
from os.path import isfile
from os.path import dirname
from pathlib import Path

global_min_note, global_max_note = 127,0
MAX_TRACK = 16

def track_to_list(s, min_note, max_note):
    ind = 0
    maxk = 0
    my_index = range(50000)
    my_columns = ["Voice 0", "Voice 1", "Voice 2", "Voice 3", "Voice 4", "Voice 5","Voice 6","Voice 7","Voice 8","Voice 9","Voice 10","Voice 11","Voice 12","Voice 13", "Voice 14", "Voice 15"]
    table = pd.DataFrame(" ", index = my_index, columns = my_columns)
    for p in s.parts:
        column = "Voice {}".format(ind)
        print(column)
        if ind<MAX_TRACK: 
            for n in p.notes:
                if n.isNote: 
                    #print(n)
                    offset = int(n.offset*4//1)
                    duration = int(n.duration.quarterLength*4//1)
                    note = n.pitch.midi+33
                    if note<min_note: min_note = note
                    if note>max_note: max_note = note
                    #print(ind, offset, duration, n.midi)
                    for i in range(offset, offset+duration):
                        line = chr(note)
                        table[column][i] = line
                        #print("table['{}'][{}] = {}".format(column, i, line))
                        if i>maxk: maxk = offset
                    #print(ind, k, table[ind, k])
            ind += 1
    #table.fillna(" ", inplace = True)
    return table, min_note, max_note, ind-1, maxk
        
def write_to_file(l, filename, length, voices):
    file = open(filename, "w")
    for r in range(length):
        lst = []
        for c in range(voices+1):
            column = "Voice {}".format(c)
            lst.append(l[column][r])
            line = ''.join(lst)+"!"
        file.write(line)
    file.write("********")
    file.close()
    
def chunks(s, n):
    """Produce `n`-character chunks from `s`."""
    for start in range(0, len(s), n):
        yield s[start:start+n]
    
def read_from_file(filename):
    my_index = range(50000)
    my_columns = ["Voice 0", "Voice 1", "Voice 2", "Voice 3", "Voice 4", "Voice 5","Voice 6","Voice 7","Voice 8","Voice 9","Voice 10","Voice 11","Voice 12","Voice 13", "Voice 14", "Voice 15"]
    table = pd.DataFrame(" ", index = my_index, columns = my_columns)
    with open(filename) as file:
        text = file.read()
        #print(text)
    file.close()
    words = "!".split(text)
    ind = 0
    for chunk in chunks(text, 4):
        #print(chunk)
        for i in range(4):
            table[i, ind]=ord(chunk[i])
        ind += 1
    return table, ind
        
    
def list_to_stream(table, maxk):
    s = stream.Score()
    for i in range(4):
        if (table[i].sum() - 320000) > 0:
            print("Voice:", i)
            p = stream.Part()
            ind = 0
            k = 0
            duration = 1
            while k < maxk:
                midi_pitch = table[i,k]-32
                if midi_pitch > 0:
                    next_midi_pitch = table[i, k+1]
                    if midi_pitch == next_midi_pitch:
                        duration += 1
                    else:
                        n = note.Note()
                        n.duration.quarterLength = duration / 4
                        n.pitch.midi = midi_pitch
                        p.append(n)
                        duration = 1
                k += 1
            s.insert(0,p)
            print("appending", p)
    return s


In [52]:
PATH = "/home/krzysztof/ml/deepbach/bach"

files = glob.glob(PATH + '/**/*.mid', recursive=True)
ind=0
bad_files = []
for file in files:
    try:
        print("Trying: ", logfilename)
        s = converter.parse(file)
        k = s.analyze('key')
        i = interval.Interval(k.tonic, pitch.Pitch('C'))
        print(k, i)
        sNew = s.transpose(i)
        kNew = sNew.analyze("key")
        print("New key: ", kNew)
    except:
        print("Skipping: ", file)
        bad_files.append(file)
    if "major" in k.name: scale = "major"
    else: scale = "minor"
    l, global_min_note, global_max_note, tracks, maxk = track_to_list(sNew, global_min_note, global_max_note)
    print(ind, file, tracks, global_min_note, global_max_note, maxk)
    txtfilename = PATH + "/txt/"+scale+"/"+str(tracks+1)+"/"+basename(file).split(".")[0]+".txt"
    print(ind, txtfilename)
    dirname = os.path.dirname(txtfilename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    write_to_file(l, txtfilename, maxk, tracks)
    ind += 1
else: print(logfilename, " exists. skipping")
    
print("Global max: {}, global min: {}, alphabet_size: {}"
      .format(global_min_note, global_max_note, global_max_note-global_min_note+1+1))

Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
0 /home/krzysztof/ml/deepbach/bach/wtcbkii/prelude6.mid 0 127 0 0
0 /home/krzysztof/ml/deepbach/bach/txt/minor/1/prelude6.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
E major <music21.interval.Interval M-3>
New key:  C major
Voice 0
1 /home/krzysztof/ml/deepbach/bach/wtcbkii/prelude9.mid 0 127 0 0
1 /home/krzysztof/ml/deepbach/bach/txt/major/1/prelude9.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
2 /home/krzysztof/ml/deepbach/bach/wtcbkii/fugue6.mid 0 127 0 0
2 /home/krzysztof/ml/deepbach/bach/txt/minor/1/fugue6.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
e- minor <music21.interval.Interval m-3>
New key:  c minor
Voice 0
3 /home/krzysztof/ml/deepbach/bach/wtcbkii/prelude8.mid 0 127 0 0
3 /home/krzysztof/ml/deepbach/bach/txt/mino

Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
D major <music21.interval.Interval M-2>
New key:  C major
Voice 0
Voice 1
32 /home/krzysztof/ml/deepbach/bach/invent/invent3.mid 1 59 117 354
32 /home/krzysztof/ml/deepbach/bach/txt/major/2/invent3.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
B- major <music21.interval.Interval m-7>
New key:  C major
Voice 0
Voice 1
33 /home/krzysztof/ml/deepbach/bach/invent/invent14.mid 1 59 117 608
33 /home/krzysztof/ml/deepbach/bach/txt/major/2/invent14.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
f minor <music21.interval.Interval P-4>
New key:  c minor
Voice 0
Voice 1
34 /home/krzysztof/ml/deepbach/bach/invent/invent9.mid 1 59 117 396
34 /home/krzysztof/ml/deepbach/bach/txt/minor/2/invent9.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
C major <music21.interval.Interval P1>
New key:  C major
Voice 0
Voice 1
35 /home/krzysztof/ml/deepbach/bach/invent/invent1.mid 1 59 117 33

Voice 1
Voice 2
61 /home/krzysztof/ml/deepbach/bach/sinfon/sinfon2.mid 2 59 117 756
61 /home/krzysztof/ml/deepbach/bach/txt/minor/3/sinfon2.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
e minor <music21.interval.Interval M-3>
New key:  c minor
Voice 0
Voice 1
Voice 2
62 /home/krzysztof/ml/deepbach/bach/sinfon/sinfon7.mid 2 59 117 516
62 /home/krzysztof/ml/deepbach/bach/txt/minor/3/sinfon7.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
g minor <music21.interval.Interval P-5>
New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
63 /home/krzysztof/ml/deepbach/bach/organ/catech6.mid 3 59 117 358
63 /home/krzysztof/ml/deepbach/bach/txt/minor/4/catech6.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
G major <music21.interval.Interval P-5>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
Voice 4
Voice 5
64 /home/krzysztof/ml/deepbach/bach/organ/schub6.mid 5 59 124 1174
64 /home/krzysztof/ml/deepbach/bach/txt/major/6/schub6.txt
Trying: 

e minor <music21.interval.Interval M-3>
New key:  c minor
Voice 0
Voice 1
Voice 2
91 /home/krzysztof/ml/deepbach/bach/organ/prefug3.mid 2 52 124 1268
91 /home/krzysztof/ml/deepbach/bach/txt/minor/3/prefug3.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
a minor <music21.interval.Interval M-6>
New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
92 /home/krzysztof/ml/deepbach/bach/chorales/10austie.mid 3 52 124 280
92 /home/krzysztof/ml/deepbach/bach/txt/minor/4/10austie.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
F major <music21.interval.Interval P-4>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
93 /home/krzysztof/ml/deepbach/bach/chorales/06christ.mid 3 52 124 128
93 /home/krzysztof/ml/deepbach/bach/txt/major/4/06christ.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
A major <music21.interval.Interval M-6>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
94 /home/krzysztof/ml/deepbach/bach/chorales/02ichdan.mid 3 52 124 

F major <music21.interval.Interval P-4>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
Voice 4
Voice 5
Voice 6
Voice 7
Voice 8
121 /home/krzysztof/ml/deepbach/bach/brandenb/brand2.mid 8 52 124 3786
121 /home/krzysztof/ml/deepbach/bach/txt/major/9/brand2.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
G major <music21.interval.Interval P-5>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
Voice 4
Voice 5
Voice 6
Voice 7
Voice 8
Voice 9
122 /home/krzysztof/ml/deepbach/bach/brandenb/brand43.mid 9 52 124 3904
122 /home/krzysztof/ml/deepbach/bach/txt/major/10/brand43.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
e minor <music21.interval.Interval M-3>
New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
Voice 4
Voice 5
Voice 6
Voice 7
Voice 8
Voice 9
123 /home/krzysztof/ml/deepbach/bach/brandenb/brand42.mid 9 52 124 852
123 /home/krzysztof/ml/deepbach/bach/txt/minor/10/brand42.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
D major

d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
150 /home/krzysztof/ml/deepbach/bach/aof/cnt1.mid 3 52 124 1424
150 /home/krzysztof/ml/deepbach/bach/txt/minor/4/cnt1.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
151 /home/krzysztof/ml/deepbach/bach/aof/reg1.mid 3 52 124 1236
151 /home/krzysztof/ml/deepbach/bach/txt/minor/4/reg1.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
152 /home/krzysztof/ml/deepbach/bach/aof/inver1.mid 3 52 124 1152
152 /home/krzysztof/ml/deepbach/bach/txt/minor/4/inver1.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
Voice 1
153 /home/krzysztof/ml/deepbach/bach/aof/can4.mid 1 52 124 1232
153 /home/krzysztof/ml/deepbach/b

B- major <music21.interval.Interval m-7>
New key:  C major
Voice 0
Voice 1
Voice 2
180 /home/krzysztof/ml/deepbach/bach/wtcbki/fugue21.mid 2 52 124 564
180 /home/krzysztof/ml/deepbach/bach/txt/major/3/fugue21.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
C major <music21.interval.Interval P1>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
181 /home/krzysztof/ml/deepbach/bach/wtcbki/fugue1.mid 3 52 124 408
181 /home/krzysztof/ml/deepbach/bach/txt/major/4/fugue1.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
E major <music21.interval.Interval M-3>
New key:  C major
Voice 0
Voice 1
Voice 2
182 /home/krzysztof/ml/deepbach/bach/wtcbki/fugue9.mid 2 52 124 448
182 /home/krzysztof/ml/deepbach/bach/txt/major/3/fugue9.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
F major <music21.interval.Interval P-4>
New key:  C major
Voice 0
Voice 1
Voice 2
183 /home/krzysztof/ml/deepbach/bach/wtcbki/fugue11.mid 2 52 124 432
183 /home/krzysztof/

e minor <music21.interval.Interval M-3>
New key:  c minor
Voice 0
Voice 1
Voice 2
210 /home/krzysztof/ml/deepbach/bach/gold/var26.mid 2 52 124 380
210 /home/krzysztof/ml/deepbach/bach/txt/minor/3/var26.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
e minor <music21.interval.Interval M-3>
New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
211 /home/krzysztof/ml/deepbach/bach/gold/var30.mid 3 52 124 380
211 /home/krzysztof/ml/deepbach/bach/txt/minor/4/var30.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
e minor <music21.interval.Interval M-3>
New key:  c minor
Voice 0
Voice 1
212 /home/krzysztof/ml/deepbach/bach/gold/var28.mid 1 52 124 380
212 /home/krzysztof/ml/deepbach/bach/txt/minor/2/var28.txt
Trying:  /home/krzysztof/ml/deepbach/bach/txt/log/prelude6.log
G major <music21.interval.Interval P-5>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
213 /home/krzysztof/ml/deepbach/bach/gold/var29.mid 3 52 124 524
213 /home/krzysztof/ml/deepbach/bach/t

In [ ]:
tracks

###### from os.path import basename

global_min_note, global_max_note = 127,0
s = converter.parse('~/ml/Untitled Folder/music_rnn/bach_clavier/partitas/all2.mid')
filename = basename('~/ml/Untitled Folder/music_rnn/bach_clavier/partitas/all2.mid').split(".")[0]+".txt"
print(filename)
l, global_min_note, global_max_note, i, maxk = track_to_list(s, global_min_note, global_max_note)

In [ ]:
127-12

In [ ]:
s.parts

In [ ]:
files

In [ ]:
import jupyter_core

In [ ]:
jrd = jupyter_core.paths.jupyter_runtime_dir()

In [ ]:
jrd